In [9]:
from kafka import KafkaProducer
from datetime import datetime
import time
import json
import requests

In [5]:
producer = KafkaProducer(bootstrap_servers='localhost:9092', key_serializer = lambda msg: json.dumps(msg).encode('utf-8'),
                            value_serializer = lambda msg: json.dumps(msg).encode('utf-8'))

In [6]:
from kafka.admin import KafkaAdminClient, NewTopic

admin_client = KafkaAdminClient(
    bootstrap_servers="localhost:9092", 
    client_id='test'
)

topic_list = []
topic_list.append(NewTopic(name="test_transit", num_partitions=2, replication_factor=1))
admin_client.create_topics(new_topics=topic_list, validate_only=False)

CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='test_transit', error_code=0, error_message=None)])

In [7]:
def fetch_data():
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def send_to_kafka(data):
    try:
        producer.send('test_transit', value=data)
    except Exception as e:
        print(f"Error sending message to Kafka: {e}")

In [10]:
url = "http://transitdata.cityofmadison.com/TripUpdate/TripUpdates.json"
while True:
    data = fetch_data()
    if data:
        send_to_kafka(data)
    time.sleep(60)

KeyboardInterrupt: 

In [13]:
producer.close()